# Try Online
If you are here to try `chainladder-python` for the first time, you are in the right place!

With the help of Google Colab, you will be able to try the package online, without any installation or additional setup.

You should use a blank copy of this workbook.
  - Open in [Google Colab](https://githubtocolab.com/casact/chainladder-python/blob/master/docs/tutorials/demo-blank.ipynb) (faster, Google Account required)
  - Open in [Binder](https://mybinder.org/v2/gh/casact/chainladder-python/master?urlpath=tree/docs/tutorials/demo-blank.ipynb) (slower, no sign up required)

It's best that you work on your version of the workbook, while comparing this site directly for additional commentary and check your solutions to the exercises.

# Setting Up
We will first need to install the package, as Google Colab's default enviroment doesn't have the chainladder package pre-installed. You will need to run this step using your terminal instead of using a python notebook when you are ready to install the package on your machine.

Simply execute `pip install chainladder`, Colab is smart enough to know that this is not a piece of python code, but to execute it in shell. FYI, `pip` stands for "Package Installer for Python".

Other commonly used packages, such as `numpy`, `pandas`, and `matplotlib` are already pre-installed, we just need to load them into our enviroment.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chainladder as cl

print("chainladder", cl.__version__)

chainladder 0.8.13


# Our Journey Begins

Let's begin by looking at a sample dataset, called `xyz`, which is hosted on https://raw.githubusercontent.com/casact/chainladder-python/master/chainladder/utils/data/xyz.csv.

Let's load the dataset into the memory with `pandas`, then inspect it.

Can you list all of the unique accident years? How many are there?

## Exercise 1
How many unique valuation years are there? Can you sort them?

```{admonition} Click to reveal the solution
:class: dropdown
`np.sort(xyz_df["DevelopmentYear"].unique())`
```

# Triangle Basics

Let's load the data into the chainladder triangle format. And let's call it `xyz_tri`.

What does the incurred triangle look like?

How about paid?

## Exercise 2
What would the Reported triangle look like?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri["Reported"]`
```

## Exercise 3
What would the Open Claim Count triangle look like?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri["Reported"] - xyz_tri["Closed"]`
```

# Pandas-like Operations

Let's see how `.iloc[...]` and `.loc[...]` simiarly to pandas. They take 4 parameters: [index, column, origin, valuation].

What if we want the row from AY 1998 Incurred data?

What if you only want the valuation at age 60?

## Exercise 4
How do we get the Incurred column at age 60?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri.iloc[:, 0, :, 4]`
```

## Exercise 5
How do we get the Incurred column at age 60, but discarding everything below origin 2005? Display the result in thousands.

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri.iloc[:, 0, 0:7, 4] / 1000`
```

## Exercise 6
How do we get the Paid values for all origins, but only the lastest 5 valuations?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri.iloc[:, 1, :, -5:]`
```

Let's use `.loc[...]` to get the incurred triangle.

## Exercise 7
What if we want the Paid value for AY 1999 at age 60?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri.loc[:, "Paid", "1999", 60]`
```

How do we get the latest diagonal only?

Very often, we want incremental triangles instead. Let's convert the Incurred triangle to the incremental form.

## Exercise 8
Can you cast the raw Incurred triangle to the incremental form, and the back to the cumulative form, and check if it mutated?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri["Incurred"].cum_to_incr().incr_to_cum() == xyz_tri["Incurred"]`
```

## Exercise 9
If we assume that the ultimate claim count is 0.25 times current Open claim count, plus the current Reported claim count, what would the ultimate claim count by origin years look like?

```{admonition} Click to reveal the solution
:class: dropdown
`(xyz_tri["Reported"] - xyz_tri["Closed"]).latest_diagonal * 0.25 + xyz_tri["Reported"].latest_diagonal`
```

We can also convert the triangle to the valuation format, what we often see on Schedule Ps.

## Exercise 10
Can you get the Incurred column for 2005 valuation?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri["Incurred"].dev_to_val().loc[:, :, :, "2005"]`
```

## Exercise 11
Can you get the Incurred columns for 2005 through 2008?

```{admonition} Click to reveal the solution
:class: dropdown
`xyz_tri["Incurred"].dev_to_val().iloc[:, :, :, 7:]`
```

Another function that is often useful is the `.heatmap()` method. Let's inspect the incurred amount and see if there are trends.

# Development

How can we get the incurred link ratios?

We can also apply a `.heatmap()` to make it too, to help us visulize the highs and lows.

Let's get a simple average LDFs for our Incurred triangle.

How about the CDFs?

## Exercise 12
Can you get the Incurred LDF but using `volume`-weighted average using all periods?

```{admonition} Click to reveal the solution
:class: dropdown
`cl.Development(average="volume").fit(xyz_tri["Incurred"]).ldf_`
```

We can also use only the latest 3 periods in the calculation of LDFs.

## Exercise 13
How do we get the Incurred LDF using simple-weighted average but with only the latest 3 periods?

```{admonition} Click to reveal the solution
:class: dropdown
`cl.Development(average="simple", n_periods=3).fit(xyz_tri["Incurred"]).ldf_`
```

## Exercise 14
Try to cast the `ldf_` to cumlative factors and back to incremental factors, did it mutate?

```{admonition} Click to reveal the solution
:class: dropdown
`cl.Development(average="simple", n_periods=3).fit(
    xyz_tri["Incurred"]
).ldf_.incr_to_cum().cum_to_incr() == cl.Development(average="simple", n_periods=3).fit(
    xyz_tri["Incurred"]
).ldf_`
```

## Exercise 15
Use the help of the [reference doc](https://chainladder-python.readthedocs.io/en/latest/modules/generated/chainladder.Development.html), see if you can figure out how to calculate the LDFs using simple average of all values, but discarding the highest and the lowest value. Did you get a warning? Make sure you understand what it means.

```{admonition} Click to reveal the solution
:class: dropdown
`cl.Development(average="simple", drop_high=True, drop_low=True).fit(
    xyz_tri["Incurred"]
).ldf_`
```

# Deterministic Models

Before we can build any models, we need to use `fit_transform()`, so that the object is actually modified with our selected development pattern(s).

Set the development of the triangle to use only 3 periods.

Let's fit a chainladder model to our Incurred triangle.

How can we get the model's ultimate estimate?

How about just the IBNR?

## Exercise 16
Can you calculate the IBNR using ultimate - the latest diagonal instead? Check it against the `ibnr_` attribute.

```{admonition} Click to reveal the solution
:class: dropdown
`cl_mod.ultimate_ - xyz_tri["Incurred"].latest_diagonal == cl_mod.ibnr_`
```

## Exercise 17
Fit the chainladder model to the paid triangle, what is the ultimate for origin year 2005?

```{admonition} Click to reveal the solution
:class: dropdown
`cl.Chainladder().fit(xyz_tri["Paid"]).ultimate_.loc[:, :, "2005", :]`
```

Let's fit an Expected Loss model, with an aprior of 90% on Premium, and get its ultimates.

Try it on the Paid triangle, do you get the same ultimate?

## Exercise 18
What would the ultimate amounts be if we assume that they are 50% of premium + $20,000?

```{admonition} Click to reveal the solution
:class: dropdown
`cl.ExpectedLoss(apriori=1.00).fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal * 0.50 + 20000
).ultimate_`
```

How about a Bornhuetter-Ferguson model?

How about Benktander, with 2 iterations?

## Exercise 19
Do you remember how many iterations of the Benktander method yields the same result as a BF model? Try to see if they reconcile.

```{admonition} Click to reveal the solution
:class: dropdown
`cl.Benktander(apriori=0.90, n_iters=1).fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal
).ultimate_ == cl.BornhuetterFerguson(apriori=0.90).fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal
).ultimate_`
```

How about Cape Cod?

Let's store the Cape Cod model as `cc_result`, so and make a bar chart over origin years to see what they look like.

## Exercise 20
With so many models built, can you plot their ultimates on a line chart, and compare their estimates? Start with the following code:

```
cl_result = cl.Chainladder().fit(xyz_tri["Incurred"])
el_result = cl.ExpectedLoss(apriori=0.90).fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal
)
bf_result = cl.BornhuetterFerguson(apriori=0.90).fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal
)
bk_result = cl.Benktander(apriori=0.90, n_iters=1).fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal
)
cc_result = cl.CapeCod().fit(
    xyz_tri["Incurred"], sample_weight=xyz_tri["Premium"].latest_diagonal
)
```

```{admonition} Click to reveal the solution
:class: dropdown
`
plt.plot(
    cl_result.ultimate_.to_frame(origin_as_datetime=True).index.year,
    cl_result.ultimate_.to_frame(origin_as_datetime=True)["2261"],
    label="Chainladder",
)
plt.plot(
    el_result.ultimate_.to_frame(origin_as_datetime=True).index.year,
    el_result.ultimate_.to_frame(origin_as_datetime=True)["2261"],
    label="Expected Loss",
)
plt.plot(
    bf_result.ultimate_.to_frame(origin_as_datetime=True).index.year,
    bf_result.ultimate_.to_frame(origin_as_datetime=True)["2261"],
    label="Bornhuetter-Ferguson",
)
plt.plot(
    bk_result.ultimate_.to_frame(origin_as_datetime=True).index.year,
    bk_result.ultimate_.to_frame(origin_as_datetime=True)["2261"],
    label="Benktander",
)
plt.plot(
    cc_result.ultimate_.to_frame(origin_as_datetime=True).index.year,
    cc_result.ultimate_.to_frame(origin_as_datetime=True)["2261"],
    label="Cape Cod",
)

plt.legend(loc="upper left")
`
```

# Stochastic Models

The Mack's Chainladder model is available.

There are many attributes that are available, such as `full_std_err_`, `total_process_risk_`, `total_parameter_risk_`, `mack_std_err_` and `total_mack_std_err_`.

MackChainladder also has a `summary_` attribute.

Let's make a graph, that shows the Paid and IBNR as stacked bars, and error bars showing Mack Standard Errors.

ODP Bootstrap is also available. Let's build sample 10,000 Incurred triangles.

We can fit a basic chainladder to all sampled triangles. We now have 10,000 simulated chainladder models, all (most) with unique LDFs.

We can use `predict()` to use the model characteristics (their unique LDFs) to predict our basic Incurred triangle.

Let's make another graph.